**Card detection - drawing a bounding box around card**

In [90]:
import cv2
import numpy as np

def clahe(img, clip_limit=2.0, grid_size=(8,8)):
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)
    return clahe.apply(img)

src = cv2.imread("img.jpeg")

# HSV thresholding to get rid of as much background as possible
hsv = cv2.cvtColor(src.copy(), cv2.COLOR_BGR2HSV)
lower_blue = np.array([0, 0, 50])
upper_blue = np.array([180, 100, 255])
mask = cv2.inRange(hsv, lower_blue, upper_blue)
result = cv2.bitwise_and(src, src, mask=mask)
b, g, r = cv2.split(result)
g = clahe(g, 5, (3, 3))

# Adaptive Thresholding to isolate the card
img_blur = cv2.blur(g, (9, 9))
img_th = cv2.adaptiveThreshold(img_blur, 0, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 51, 2)

contours, hierarchy = cv2.findContours(img_th,
                                           cv2.RETR_CCOMP,
                                           cv2.CHAIN_APPROX_SIMPLE)

# Filter the rectangle by choosing only the big ones
# and choose the brightest rectangle as the card
max_brightness = 0
canvas = src.copy()
for cnt in contours:
    rect = cv2.boundingRect(cnt)
    x, y, w, h = rect
    if w*h > 40000 and w*h<45000:
        mask = np.zeros(src.shape, np.uint8)
        mask[y:y+h, x:x+w] = src[y:y+h, x:x+w]
        brightness = np.sum(mask)
        if brightness > max_brightness:
            brightest_rectangle = rect
            max_brightness = brightness
        # cv2.imshow("mask", mask)
        cv2.waitKey(0)

x, y, w, h = brightest_rectangle
print("The width of card is", w)
cv2.rectangle(canvas, (x, y), (x+w, y+h), (0, 255, 0), 5)
# cv2.imshow("canvas", canvas)
canvas = cv2.circle(canvas, (x+int(w/2)-10,y+int(h/2)+50), 5, color=(0, 0, 255), thickness=-1)
cv2.imwrite("card_detected.jpg", canvas)
cv2.waitKey(0)

The width of card is 493


-1

In [91]:
crop=canvas[x:x+h,y:y+w]
cv2.imwrite("card.jpg", crop)

True

**Refining the card size**

In [92]:
import cv2
import numpy as np


card_img = cv2.imread("card.jpg")
orig_img = card_img
card_img = cv2.cvtColor(card_img, cv2.COLOR_BGR2GRAY)
threshold = int(np.average(card_img))

threshold_img = cv2.adaptiveThreshold(card_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 199, 2)
colors, counts = np.unique(threshold_img.flatten(), 
                           return_counts = True, 
                           axis = 0)

area = 0

if(colors[0] == 255):
    area = counts[0]

else:
    area = counts[1]

width = int(np.sqrt(area/28)*7)
height = int(np.sqrt(area/28)*4)
width_final=width
print("The corrected width of card is", width_final)
centroid = np.array((int(threshold_img.shape[1]/2), int(threshold_img.shape[0]/2)))


cv2.rectangle(orig_img, centroid + np.array([width/2, height/2], dtype=np.int16), centroid - np.array([width/2, height/2], dtype=np.int16), color=(0, 0,255), thickness=5)

cv2.imwrite("threshold_image.jpg", orig_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

The corrected width of card is 441


**Progressive segment height**

In [108]:
import cv2
import numpy as np

img = cv2.imread('img.jpeg')
img_resized = img
# img_resized = cv2.resize(img, (0,0), fx=0.5, fy=0.5)
gray_resized = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)

t_lower = 50  # Lower Threshold
t_upper = 150  # Upper threshold
  
# Applying the Canny Edge filter
edge = cv2.Canny(img, t_lower, t_upper)
# edge_resized = cv2.resize(edge, (0,0), fx=0.5, fy=0.5)
edge_resized = edge

classifier =cv2. CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
boxes = classifier.detectMultiScale(gray_resized)
pupil=[]
w=0
h=0
for box in boxes:
    x, y, width, height = box
    x2, y2 = x + width, y + height
    # cv2.rectangle(img_resized, (x, y), (x2, y2), (0,255,0), 5)
    pupil.append([int((x+x2)/2), int((y+y2)/2)])
    w=width
    h=height

search_region = edge_resized[int(pupil[0][1]):int(pupil[0][1]+424/2), int(pupil[0][0]-w/2):int(pupil[0][0]+w/2)]
element_wise_avg = []
max_idx = 0
max_val = 0
for i in range(search_region.shape[0]):
    average = np.average(search_region[i])
    element_wise_avg.append(average)
    if average > max_val:
        max_val = average
        max_idx = i



pupil = np.array(pupil)
pupil = pupil
line =cv2.line(img_resized, tuple(pupil[0]), tuple([pupil[0][0], pupil[0][1] + max_idx]), (0, 255, 0), thickness=5)
image = cv2.circle(img_resized, (int(pupil[0][0]),int(pupil[0][1])), 5, color=(0, 0, 255), thickness=-1)
image = cv2.circle(img_resized, (int(pupil[1][0]),int(pupil[1][1])), 5, color=(0, 0, 255), thickness=-1)

line = cv2.putText(line, str(round((9*max_idx)/width_final,2))+"cm", tuple([pupil[0][0],pupil[0][1]+int(max_idx/2)]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5, cv2.LINE_AA)
print("The progressive segment height is", max_idx)
cv2.imwrite("line.jpg", line)
cv2.waitKey(0)
cv2.destroyAllWindows()

The progressive segment height is 113


**Final Scaling for progressive segment height**

In [94]:
print("The progressive segment height in cm is", (9*max_idx)/width_final)

The progressive segment height in cm is 2.306122448979592


**Bifocal segment height**

In [95]:
import numpy as np
import dlib
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import statistics

In [109]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
img = dlib.load_rgb_image("img.jpeg")    
rect = detector(img)[0]
sp = predictor(img, rect)
landmarks = np.array([[p.x, p.y] for p in sp.parts()])
pupil_edge_x = []
pupil_edge_y = []
for i in [41,47]:
        pupil_edge_x.append(landmarks[i][0])
        pupil_edge_y.append(landmarks[i][1])
        
x_min = min(pupil_edge_x)
x_max = max(pupil_edge_x)

y_min = min(pupil_edge_y)
y_max = max(pupil_edge_y)
img = cv2.imread('img.jpeg')
img = cv2.circle(img, (x_min,y_min+10), 5, color=(0, 0, 255), thickness=-1)
img =cv2.line(img, (x_min,y_min+10), tuple([x_min, pupil[1][1] + max_idx]), (0, 255, 0), thickness=5)
img = cv2.putText(img, str(round((9*(pupil[1][1]+max_idx-y_min-10))/width_final,2))+"cm", tuple([x_min,int((y_min+10+pupil[1][1]+max_idx)/2)]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5, cv2.LINE_AA)
print("The bifocal segment height is", pupil[1][1]+max_idx-y_min-10)
cv2.imwrite("bifocal.jpg",img)

The bifocal segment height is 90


True

In [105]:
print("The bifocal segment height in cm is", (9*(pupil[1][1]+max_idx-y_min-10))/width_final)

The bifocal segment height in cm is 1.836734693877551
